In [1]:
#Importing drive model from the google.colab package
from google.colab import drive

#Mounting the google drive to a specific path
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/GitHub_Projects/FTSE_350_Anomaly_Detection

/content/drive/MyDrive/GitHub_Projects/FTSE_350_Anomaly_Detection


In [3]:
# Configuring Git user details
!git config --global user.email "dorothy.sarpongk@gmail.com"
!git config --global user.name "01DorSarpong"

In [4]:
!git add Unsupervised_Anomaly_Detection.ipynb

In [5]:
!git commit -m "Adding current state of Unsupervised_Anomaly_Detection.ipynb after repository reset"

[main d4e5150] Adding current state of Unsupervised_Anomaly_Detection.ipynb after repository reset
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite Unsupervised_Anomaly_Detection.ipynb (99%)


In [8]:
!git add .

In [9]:
!git commit -m "trying to fix this git status issue"

[main 9f66928] trying to fix this git status issue
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite Unsupervised_Anomaly_Detection.ipynb (83%)


In [ ]:
# Make sure you are in your repository's directory
%cd /content/drive/MyDrive/GitHub_Projects/FTSE_350_Anomaly_Detection

# Stage your latest changes to the notebook
!git add Unsupervised_Anomaly_Detection.ipynb

# Commit these changes
!git commit -m "Committed newest changes after previous push attempt"

In [ ]:
# Importing libraries for code

import pandas as Pd
import numpy  as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from tqdm import tqdm


In [ ]:
def download_and_save_FTSE_stocks(tickers: list, start_date: str, end_date: str, directory: str):
  """
  This function downloads historical stock data for a list of tickers and saves it to CSV files.

  Args:
    tickers_list (list): A list of stock ticker symbols (e.g., ['TSCO.L', 'BARC.L']).
    start_date (str): The start date for data download in 'YYYY-MM-DD' format.
    end_date (str): The end date for data download in 'YYYY-MM-DD' format.
    directory (str): The path to the directory where CSV files will be saved.
  """

  # Ensure the save directory exists
  if not os.path.exists(directory):
    os.makedirs(directory)
    print(f"Created directory: {directory}")

  print(f"Starting download for {len(tickers)} tickers from {start_date} to {end_date}...")

  for ticker in tqdm(tickers, desc="Downloading Stocks"):
    # Format the filename: remove '.L' and add date range for clarity
    cleaned_ticker = ticker.replace('.L', '')
    file_name = f"{cleaned_ticker}_{start_date.replace('-', '')}_{end_date.replace('-', '')}.csv"
    full_file_path = os.path.join(directory, file_name)

    try:
      df = yf.download(ticker, start=start_date, end=end_date, auto_adjust=False)

      if not df.empty:
        df.to_csv(full_file_path)
        # print(f"✅ Saved data for {ticker} to {full_file_path}") # Optional: uncomment for more verbose output
      else:
        print(f"⚠️ No data available for {ticker} for the specified period.")
    except Exception as e:
      print(f"❌ Error downloading or saving data for {ticker}: {e}")

  print("Download process completed.")


In [ ]:
#Creating a list of FTSE 100 and FTSE 250 tickers

FTSE_100_tickers = ["AZN.L", "HSBA.L", "ULVR.L", "REL.L", "BATS.L", "BP.L", "GSK.L", "DGE.L",
                   "RR.L", "NG.L", "BARC.L", "TSCO.L", "PRU.L", "BHP.L", "BT-A.L",]

FTSE_250_tickers = ["BWY.L", "EMG.L", "JUST.L", "SXS.L", "CKN.L", "LRE.L", "RAT.L", "THG.L",
                    "JDW.L", "SCT.L", "DOM.L", "SRE.L", "HIK.L", "ICGT.L", "HSX.L"]

In [ ]:
#Defining the period for stocks range
start_date = "2014-01-01"
end_date = "2024-12-31"

In [ ]:
# Defining the path to save the CSVs

ftse_100_path = '/content/drive/MyDrive/GitHub_Projects/FTSE_350_Anomaly_Detection/FTSE_100'
ftse_250_path = '/content/drive/MyDrive/GitHub_Projects/FTSE_350_Anomaly_Detection/FTSE_250'


In [ ]:
# Calling the function for FTSE 100 and FTSE 250 tickers

download_and_save_FTSE_stocks(
    tickers=FTSE_100_tickers,
    start_date=start_date,
    end_date=end_date,
    directory=ftse_100_path
)

download_and_save_FTSE_stocks(
    tickers=FTSE_250_tickers,
    start_date=start_date,
    end_date=end_date,
    directory=ftse_250_path
)

In [ ]:
!git add Unsupervised_Anomaly_Detection.ipynb
!git commit --amend --no-edit

[main 35cb409] Refactored stock download function
 Date: Fri Jun 13 23:13:00 2025 +0000
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite Unsupervised_Anomaly_Detection.ipynb (89%)


In [ ]:
!git status

On branch main
Your branch is ahead of 'origin/main' by 2 commits.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Unsupervised_Anomaly_Detection.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
%cd /content/drive/MyDrive/GitHub_Projects/FTSE_350_Anomaly_Detection

/content/drive/MyDrive/GitHub_Projects/FTSE_350_Anomaly_Detection


In [ ]:

!git add Unsupervised_Anomaly_Detection.ipynb


!git commit -m "Commit latest changes after previous pushes"

[main 584799e] Commit latest changes after previous pushes
 1 file changed, 1 insertion(+), 1 deletion(-)


In [ ]:
!git status

On branch main
Your branch is ahead of 'origin/main' by 3 commits.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Unsupervised_Anomaly_Detection.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git status

On branch main
Your branch is ahead of 'origin/main' by 3 commits.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Unsupervised_Anomaly_Detection.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
